In [ ]:

!pip install SQLAlchemy==1.4.46
!pip install pandasql

In [ ]:
import pandasql as pdsql
import pandas as pd

In [ ]:
# Load in raw table
event = pd.read_csv('Event.csv')

In [ ]:
event

In [ ]:
event.columns

In [ ]:
event["device_id"].unique()[10]

In [ ]:
event1 = event[event["device_id"] == "6cb264f7-86b4-43ca-949f-11d3264fc60d"]
event1

In [ ]:
event1.info()

In [ ]:
sum1 = event1["revenue"].sum()
sum1

In [ ]:
# Runing this cell using Ctrl+Enter to define a SQL function
def run_sql(sql_string):
    try:
        result = pdsql.sqldf(sql_string)
        return result
    except Exception as e:
        print('''Query Excecution Failed''')
        print('Error: ',e)

def to_csv(table, target_file_name):
    table.to_csv(target_file_name+'.csv')

### Question 1


#### An "Event" is a snapshot of event that happened in game, for example, when user logs in the game, an event 'session_start' is being sent, when user conducts a purchase, 'iap_purchase' is being sent in this case.
- device_id: each device_id indicates a unique device
- event_ts: timestamp of the event happening
- event_name: name of the event
- platform: whether it's iOS or Android
- session_id: all events happened in a session shares the same session ID
- event_location: where in game this event is happening
- session_duration: how long has session started when this event happen, in seconds
- country: country of the user
- revenue: How much has this user spend, in USD
- item_purchase: what in game item is being purchased
- Ad_type: The game will show ads to players in game, player can get an in game item as a reward and the game gets ad revenue. This is the type of Advertisement being shown to the user in game
- Ad_placement: where in the game is this ad being placed

#### Observe event table, what insight can you find about revenue?

To Calculate Total Revenue Generated


In [ ]:
run_sql('''SELECT sum(Revenue) As Total_revenue FROM event''')

Total Revenue Generated per event

In [ ]:
run_sql('''SELECT event_name, sum(revenue) FROM event Group by event_name''' )

Revenue Generated By Platform in Ascending Order

In [ ]:
run_sql('''SELECT platform, SUM(revenue) AS platform_revenue,
       (SUM(revenue) * 100) / (SELECT SUM(revenue) FROM event) AS percentage_total_revenue
FROM event
GROUP BY platform
Order by sum(revenue) ASC''')



Total Sum Of Revenue Generated according to item_purchased in Descending Order

In [ ]:
run_sql('''SELECT item_purchased, sum(revenue) From event
Group by item_purchased
Order by sum(revenue) DESC''')

Calculating Total Revenue Generated in Each Location

In [ ]:
run_sql('''SELECT country, sum(revenue)
FROM event
GROUP by country
Order by sum(revenue) DESC''')

Top Five Unique Device With The Highest Revenue And Countries

In [ ]:
run_sql(''' SELECT Distinct(device_id), sum(revenue), country from event
Group by device_id
Order by sum(revenue) DESC
Limit 5''')

Average Revenue Generated

In [ ]:
run_sql('''SELECT AVG(revenue) AS average_revenue
FROM (
    SELECT device_id, SUM(revenue) AS revenue
    FROM event
    GROUP BY device_id
) AS user_revenue
''' )

#### What's the average number of events triggered per user?

Average Number Of Event Per Users

In [ ]:
run_sql(''' SELECT COUNT (*) / COUNT(DISTINCT device_id) AS average_events_per_user
FROM event''')

### Question 2
##### Creating a table called device, with columns defined below, this is a table which designed to store user level data

##### - LTV
- Life time revenue of a user

##### - Payer Type, defined below, according to LTV:
- \$0 Non Payer
- between \$0 and \$40: Minnow
- between \$40 and \$80: Dolphin
- between \$80 and \$120: Whale
- over \$120: Super Whale


##### - Install Date
- Install date is defined as the date when first event is sent

##### - Unique days active

##### - Country

##### - Platform


Creating The Table 'Device' From Already Existing Table 'Event'

In [ ]:
device = run_sql('''select device_id,
SUM(revenue) AS LTV,
substr(event_ts, 1, 10) AS install_date,
COUNT(DISTINCT SUBSTR(event_ts, 1, 10)) AS unique_active_days, platform,
country
from event
group by device_id''')


Using The Common Table Expression, I Defined my New Columns According To The Given Definition.



In [ ]:
device = run_sql('''WITH cte AS (
    SELECT
        device_id,
        SUM(revenue) AS LTV,
        substr(event_ts, 1, 10) AS install_date,
        COUNT(DISTINCT SUBSTR(event_ts, 1, 10)) AS unique_active_days,

        platform,
        country
        FROM
        event
        GROUP BY
        device_id)
SELECT
device_id,
install_date,
unique_active_days,
platform,
country,
    LTV,
    CASE
        WHEN LTV = 0 THEN 'Non Payer'
        WHEN LTV BETWEEN 0 AND 40 THEN 'Minnow'
        WHEN LTV BETWEEN 40 AND 80 THEN 'Dolphin'
        WHEN LTV BETWEEN 80 AND 120 THEN 'Whale'
        ELSE 'Super Whale'
    END AS payer_type
FROM
    cte
''')

Querying The New Table 'Device'.

In [ ]:
run_sql('''select * from device''')

Counting The Numbers of the users on payer_type 'Minnow'

In [ ]:
run_sql(''' SELECT count(device_id) From device where payer_type = 'Minnow' ''')

Counting The Numbers of the users on payer_type 'Super_Whale'

In [ ]:
run_sql(''' SELECT count(device_id) From device where payer_type = 'Super Whale' ''')

Counting The Numbers of the users on payer_type 'Dolphin'

In [ ]:
run_sql(''' SELECT count(device_id) From device where payer_type = 'Dolphin' ''')

### Question3: Retention
#### -  Retention is defined as number of user returned X days after they install
For example: If 100 users installed on Jan 1st (Day0), 50 came back on Jan 2nd (day1), 30 came back on Jan 4th(day3), then day1 retention is 50/100 = 50%, day3 is 30/100 = 30%

Given above, calculate Day1,3,7,30 retention for each install date,

__Note__: Retention 0 and retention NULL is very different! For example, suppose today is Jan 5th, then day7 retention won't be avaialble for people installed on Jan 1st, since there's only 5 days passed. As such, Day7 retention should be set to NULL, rather than 0. Whereas if nobody returns on Jan 3rd, then day2 retention is 0, rather then null.

Write a query to calculate retention by install_date.

Calculating Retention for Day 1,3,7 and 30.

In [ ]:

run_sql('''SELECT
    install_date,
    COUNT(DISTINCT Device_id) AS total_installs,
    COUNT(CASE WHEN unique_active_days >= 1 THEN Device_id END) AS day1_retention,
    COUNT(CASE WHEN unique_active_days >= 3 THEN Device_id END) AS day3_retention,
    COUNT(CASE WHEN unique_active_days >= 7 THEN Device_id END) AS day7_retention,
    COUNT(CASE WHEN unique_active_days >= 30 THEN Device_id END) AS day30_retention
FROM
    device
GROUP BY
    install_date''')


Calculating the Retention In Percentage

In [ ]:
run_sql('''SELECT
    install_date,
    COUNT(DISTINCT Device_id) AS total_installs,
    ROUND(CAST(COUNT(CASE WHEN unique_active_days >= 1 THEN Device_id END) AS FLOAT) / COUNT(DISTINCT Device_id) * 100, 2) AS day1_retention_percentage,
    ROUND(CAST(COUNT(CASE WHEN unique_active_days >= 3 THEN Device_id END) AS FLOAT) / COUNT(DISTINCT Device_id) * 100, 2) AS day3_retention_percentage,
    ROUND(CAST(COUNT(CASE WHEN unique_active_days >= 7 THEN Device_id END) AS FLOAT) / COUNT(DISTINCT Device_id) * 100, 2) AS day7_retention_percentage,
    ROUND(CAST(COUNT(CASE WHEN unique_active_days >= 30 THEN Device_id END) AS FLOAT) / COUNT(DISTINCT Device_id) * 100, 2) AS day30_retention_percentage
FROM
    device
GROUP BY
    install_date''')
